In [1]:
!pip install virtualenv
!mkdir myenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 24.1 MB/s eta 0:00:00


In [2]:
!virtualenv myenv
!source myenv/bin/activate

created virtual environment CPython3.10.12.final.0-64 in 928ms
  creator CPython3Posix(dest=/content/myenv, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==23.2.1, setuptools==68.0.0, wheel==0.41.0
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


In [3]:
!pip install transformers[torch]
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 31.4 MB/s eta 0:00:00


In [4]:
!pip install transformers torch python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 52.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184489 sha256=f64f90549119b37300ec05d4b8d1e5491f55ab35deeb123a6a92d6277d23bf1e
  Stored in directory: /root/.cache/pip/wheels/80/27/06/837436d4c3bd989b957a91679966f207bfd71d358d63a8194d
Successfully built python-docx


In [5]:
!pip install torch

In [25]:
import torch
from transformers import BertForSequenceClassification, BertTokenizer, Trainer, TrainingArguments
from torch.utils.data import Dataset
from docx import Document
# Custom dataset class
class IntentDataset(Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels

    def __len__(self):
        return len(self.inputs['input_ids'])

    def __getitem__(self, idx):
        return {
            'input_ids': self.inputs['input_ids'][idx],
            'attention_mask': self.inputs['attention_mask'][idx],
            'label': self.labels[idx]
        }

# Load pre-trained model and tokenizer
model_name = 'bert-base-uncased'
num_labels = 1  # One label for technical
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
tokenizer = BertTokenizer.from_pretrained(model_name)

# Load and extract text from the .docx file
docx_file_path = r'/content/intents.docx'
doc = Document(docx_file_path)
technical_intents = [paragraph.text for paragraph in doc.paragraphs if paragraph.text.strip() != ""]

# Preprocess the extracted text
inputs = tokenizer(technical_intents, padding=True, truncation=True, return_tensors="pt")
labels = torch.tensor([0.0] * len(technical_intents), dtype=torch.float)  # Use floating-point labels

# Define custom dataset
train_dataset = IntentDataset(inputs, labels)

# Fine-tuning arguments
training_args = TrainingArguments(
    per_device_train_batch_size=8,
    output_dir="./intent_classification_model",
    num_train_epochs=3,
    learning_rate=1e-5,
    evaluation_strategy="no",
    do_eval=False,
)

# Define Trainer with the custom dataset
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

# Fine-tune the model
trainer.train()

# Example inference (similar to previous code)
user_input = "How do convolutional neural networks work?"
user_inputs = tokenizer(user_input, padding=True, truncation=True, return_tensors="pt").input_ids.to('cuda')

device = torch.device("cuda")
model.cuda()

with torch.no_grad():
    logits = model(**user_inputs).logits

predicted_label = int(logits > 0.5)  # Threshold the logits
predicted_intent = "technical" if predicted_label == 0 else "non-technical"

print(f"Predicted Intent: {predicted_intent}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,0.007000


TypeError: ignored